In [ ]:
# Imports.
import os
import numpy as np
import soundfile as sf
import matplotlib.pyplot as plt
import params as yamnet_params
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers, regularizers 
import yamnet as yamnet_model
import params as yamnet_params
import features as features_lib
import pandas as pd
# Set random seed for TensorFlow
tf.random.set_seed(42)
np.random.seed(42)

2025-05-22 00:51:19.724885: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-22 00:51:20.549225: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-22 00:51:21.940491: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
import params as yamnet_params
import features as features_lib
import yamnet as yamnet_model
sr = 10000
params = yamnet_params.Params(sample_rate=sr, patch_hop_seconds=0.1)
yamnet=yamnet_model.yamnet_frames_model(params)
class_names = yamnet_model.class_names('yamnet_class_map_1.csv')

(None,)


2025-03-24 23:26:00.262675: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2025-03-24 23:26:00.262737: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: user
2025-03-24 23:26:00.262751: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: user
2025-03-24 23:26:00.262973: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 535.183.1
2025-03-24 23:26:00.263013: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 535.183.1
2025-03-24 23:26:00.263024: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 535.183.1


In [ ]:
#load pre-trained weights 
import tensorflow as tf
# Create a dummy input
dummy_input = tf.random.normal([1, 103,98])

# Get output before loading weights
output_before = yamnet(dummy_input)

layer_to_skip = 'dense_1'  # Replace with the actual layer name

# Load weights for all layers
try:
    yamnet.load_weights('yamnet_ndata.h5', by_name=True)
except Exception as e:
    print(f"Error loading weights: {e}")

# Manually handle the problematic layer
try:
    layer = yamnet.get_layer(name=layer_to_skip)
    # Optionally set weights to some default values or random
    layer.set_weights([np.random.randn(*w.shape) for w in layer.weights])
except Exception as e:
    print(f"Error setting weights for layer {layer_to_skip}: {e}")

# Get output after loading weights
output_after = yamnet(dummy_input)

# Compare outputs
if not np.allclose(output_before.numpy(), output_after.numpy()):
    print("Model output has changed after loading weights.")
else:
    print("Model output has not changed.")


Error setting weights for layer dense_1: No such layer: dense_1. Existing layers are: ['input_1', 'tf.reshape', 'input_2', 'layer1/conv', 'layer1/conv/bn', 'layer1/relu', 'layer2/depthwise_conv', 'layer2/depthwise_conv/bn', 'layer2/depthwise_conv/relu', 'layer2/pointwise_conv', 'layer2/pointwise_conv/bn', 'layer2/pointwise_conv/relu', 'layer3/depthwise_conv', 'layer3/depthwise_conv/bn', 'layer3/depthwise_conv/relu', 'layer3/pointwise_conv', 'layer3/pointwise_conv/bn', 'layer3/pointwise_conv/relu', 'layer4/depthwise_conv', 'layer4/depthwise_conv/bn', 'layer4/depthwise_conv/relu', 'layer4/pointwise_conv', 'layer4/pointwise_conv/bn', 'layer4/pointwise_conv/relu', 'layer5/depthwise_conv', 'layer5/depthwise_conv/bn', 'layer5/depthwise_conv/relu', 'layer5/pointwise_conv', 'layer5/pointwise_conv/bn', 'layer5/pointwise_conv/relu', 'layer6/depthwise_conv', 'layer6/depthwise_conv/bn', 'layer6/depthwise_conv/relu', 'layer6/pointwise_conv', 'layer6/pointwise_conv/bn', 'layer6/pointwise_conv/relu',

In [8]:
# Freeze all layers
for layer in yamnet.layers:
    layer.trainable = False
for i, layer in enumerate(yamnet.layers):
    if 'layer13' in layer.name or 'layer14' in layer.name:
        layer.trainable = True

In [ ]:
import os
import numpy as np
import scipy.io.wavfile as wav
from scipy import signal
from concurrent.futures import ProcessPoolExecutor
from tensorflow.keras.utils import to_categorical
from scipy.signal import spectrogram

np.random.seed(42)
# Specify the folder name
dataset_folder = '/DATA-CRPS/Audio_Files_and_MatLab_Code_used_for_Advance_Prediction_of_CHF/Audio_files'

# Create a list of audio files with labels
commands = ["BKG", "NB","TB"]
audio_files = []
labels = []

for root, _, files in os.walk(dataset_folder):
    for file in files:
        if file.endswith('.wav'):
            label = os.path.basename(root)
            if label in commands:
                audio_files.append(os.path.join(root, file))
                labels.append(label)

# Shuffle and partition the data into training, validation, and testing sets
indices = np.random.permutation(len(audio_files))

n_files = len(audio_files)
n_valid = n_files // 5
n_test = n_files // 5

valid_indices = indices[:n_valid]
test_indices = indices[n_valid:n_valid + n_test]
train_indices = indices[n_valid + n_test:]

audio_files_valid = [audio_files[i] for i in valid_indices]
audio_files_test = [audio_files[i] for i in test_indices]
audio_files_train = [audio_files[i] for i in train_indices]

valid_labels = [labels[i] for i in valid_indices]
test_labels = [labels[i] for i in test_indices]
train_labels = [labels[i] for i in train_indices]

# Audio Feature Extraction
fs = 10000  # Known sample rate of the data set
segment_duration = 1
frame_duration = 0.025
hop_duration = 0.010
fft_size=512
segment_samples = int(segment_duration * fs)
frame_samples = int(frame_duration * fs)
hop_samples = int(hop_duration * fs)
min_frequency = 0
max_frequency = 2000

# Function to extract features from an audio file
def extract_features(file):
    sample_rate, y = wav.read(file)
    f, t, Zxx = spectrogram(y, fs=fs, nperseg=frame_samples, noverlap=frame_samples-hop_samples,nfft = 512)#window='hann'
    freq_mask = (f >= min_frequency) & (f <= max_frequency)
    freq_mask2 = (f >= 0) & (f <= 50)
    linear_spectrum_db = Zxx.astype(float) #np.abs(Zxx.astype(float))
    # linear_spectrum_db[freq_mask2, : ] = 0
    linear_spectrum_db = linear_spectrum_db[freq_mask, : ]
    
    # linear_spectrum_db = np.log10(np.abs(Zxx)+1e-6)[freq_mask, : ]
    return linear_spectrum_db[..., np.newaxis]

# Extract features in parallel
def process_files(file_list):
    features_list = [extract_features(file) for file in file_list]
    return features_list
features = extract_features(audio_files_train[0])
print(features.shape)
[num_hops, num_features,_] = features.shape
print(num_hops,num_features)

train_features = np.zeros((len(audio_files_train),num_hops,num_features,1))
valid_features = np.zeros((len(audio_files_valid),num_hops,num_features,1))
test_features = np.zeros((len(audio_files_test),num_hops,num_features,1))

# Parallel processing
num_workers =  10 # Number of workers should not exceed the number of files
with ProcessPoolExecutor(max_workers=num_workers) as executor:
    train_features_list = list(executor.map(extract_features, audio_files_train))
    valid_features_list = list(executor.map(extract_features, audio_files_valid))
    test_features_list = list(executor.map(extract_features, audio_files_test))

for idx, features in enumerate(train_features_list):
    train_features[idx,:, :, :] = features

for id, features in enumerate(valid_features_list):
    valid_features[id,:, :, :] = features

for i, features in enumerate(test_features_list):
    test_features[i,:, :, :] = features


# Normalization
un_norm = 2 / (np.sum(signal.windows.hann(frame_samples, sym=False))**2)

train_features /= un_norm
valid_features /= un_norm
test_features /= un_norm

epsil = 1e-6
train_features = np.log10(train_features + epsil)
valid_features = np.log10(valid_features + epsil)
test_features = np.log10(test_features + epsil)

print("Training features shape:", train_features.shape)
print("Validation features shape:", valid_features.shape)
print("Testing features shape:", test_features.shape)

label_map = {label: idx for idx, label in enumerate(commands)}
train_labels_cat = to_categorical([label_map[label] for label in train_labels])
valid_labels_cat = to_categorical([label_map[label] for label in valid_labels])
test_labels_cat = to_categorical([label_map[label] for label in test_labels])

In [ ]:
# Train the model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping,Callback
from sklearn.utils.class_weight import compute_class_weight
seed_value = 42
tf.random.set_seed(seed_value)
# Compute class weights
train_labels_numeric = np.argmax(train_labels_cat, axis=1)
class_weights_array = compute_class_weight('balanced', classes=np.unique(train_labels_numeric), y=train_labels_numeric)
class_weights = dict(enumerate(class_weights_array))

early_stopping = EarlyStopping(monitor='val_accuracy', patience=15, restore_best_weights=True, mode='max')
optimizer = Adam(learning_rate=0.0001)
yamnet.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=optimizer)


epochs = 25
batch_size = 128
history = yamnet.fit(train_features, train_labels_cat,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(valid_features, valid_labels_cat),
                    class_weight=class_weights,
                    callbacks=[early_stopping],
                    verbose=1)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
# Evaluate the model on training data
train_loss, train_accuracy = yamnet.evaluate(valid_features, valid_labels_cat, verbose=1)
print(f"Val loss: {train_loss:.4f}")
print(f"Val accuracy: {train_accuracy:.4f}")
train_loss, train_accuracy = yamnet.evaluate(test_features, test_labels_cat, verbose=1)
print(f"TEst loss: {train_loss:.4f}")
print(f"TEst accuracy: {train_accuracy:.4f}")
test_pred = np.argmax(yamnet.predict(test_features), axis=1)
# Compute confusion matrix
cm = confusion_matrix(np.argmax(test_labels_cat, axis=1), test_pred)
labels= ['BKG','NB','TB']
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
# Plot the confusion matrix
fig, ax = plt.subplots(figsize=(10, 10))
# Customizing font size
disp.plot(ax=ax, cmap=plt.cm.Blues)
plt.title('Confusion Matrix for Validation Data', fontsize=16)
plt.xlabel('Predicted labels', fontsize=18)
plt.ylabel('True labels', fontsize=18)
# Increase font size for ticks
ax.tick_params(axis='both', which='major', labelsize=16)

# Increase font size for numbers in the matrix
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        # Adjust the text position and size to avoid overlap
        value = cm[i, j]
        ax.text(j, i, f'{value}', ha='center', va='center', fontsize=14, color='black', fontweight='bold', bbox=dict(facecolor='lightblue', alpha=0.5))
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

# Compute core data
cm = confusion_matrix(train_labels_class, train_pred)
report = classification_report(train_labels_class, train_pred, target_names=commands, output_dict=True)
classes = commands
n = len(classes)

# Extract per-class metrics
precision = [report[c]['precision'] for c in classes]
recall = [report[c]['recall'] for c in classes]
f1 = [report[c]['f1-score'] for c in classes]
accuracy_per_class = cm.diagonal() / cm.sum(axis=1)

# Create a padded matrix with 2 extra rows and 2 extra columns
# Layout: (top row for F1, bottom row for accuracy)
#         (left column for precision, right for accuracy per class)
full_matrix = np.full((n + 2, n + 2), '', dtype=object)

# Insert confusion matrix into center (only this part will be visualized as a heatmap)
for i in range(n):
    for j in range(n):
        full_matrix[i + 1, j + 1] = cm[i, j]


# Labels for axes
xtick_labels = [''] + classes + ['']
ytick_labels = [''] + classes + ['']

# Plot the confusion matrix with a heatmap
plt.figure(figsize=(4, 3))
fontsize = 11
ax = sns.heatmap(cm, annot=True, fmt='d', cmap="Purples", cbar=False, cbar_kws={'label': 'Count'},annot_kws={"size": fontsize},
                 xticklabels=classes, yticklabels=classes, linewidths=0.5, square=True)
ax.tick_params(axis='both', which='both', length=0)
ax.tick_params(axis='y', pad=17)
ax.tick_params(axis='x', pad=16)
# plt.ylabel(" True", 
#            fontsize=12, 
#            color='white',
#            bbox=dict(facecolor='green', edgecolor='none', boxstyle='round,pad=0.2'))

# plt.xlabel("Predicted", 
#            fontsize=12, 
#            bbox=dict(facecolor='red', edgecolor='none', boxstyle='round,pad=0.2'), 
#            color='white')



# # Add the precision, recall, F1, and accuracy annotations around the matrix
for j in range(n):
    ax.text(j + 0.5, -0.1, f"F1: {f1[j]:.2f}", ha='center', va='center', fontsize=11, color='black')
    
for j in range(n):  # Known sample rate of the data set
    ax.text(j + 0.5, n + 0, f"A: {accuracy_per_class[j]:.2f}", ha='center', va='center', fontsize=11, color='black')

for i in range(n):
    ax.text(-0.1, i + 0.5, f"P: {precision[i]:.2f}", ha='center', va='center', fontsize=11, color='black',rotation=90)

for i in range(n):
    ax.text(n + 0.1, i + 0.5, f"R: {recall[i]:.2f}", ha='center', va='center', fontsize=11, color='black',rotation=90)

# Add titles and labels
# plt.title("Confusion Matrix with Precision (Left), Recall (Right), F1 (Top), Accuracy (Bottom)")
plt.xticks(rotation=0,fontsize=11)
plt.yticks(rotation=90,fontsize=11)
plt.tight_layout()
plt.show()